In [1]:
import mysql.connector
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import time

Creating new mysql connection and cursor to database

In [2]:
def make_connection():
    global cnx
    global cursor
    
    cnx = mysql.connector.connect(
      host="localhost",
      user="root",
      password="root",
      database="court_case_journal",
      auth_plugin="mysql_native_password"
    )
    cursor = cnx.cursor()

In [3]:
def break_connection():
    cursor.close()
    cnx.close()

Insert PDF file data

In [4]:
def insert_record(filename, textdata):
    #SQL command
    add_file = ("INSERT INTO pdf "
               "(filename, textdata) "
               "VALUES (%s, %s)")
    args = (filename, textdata)


    cursor.execute(add_file, args)

    if cursor.lastrowid:
        msg = "last insert id {fileid}: {filename}".format(fileid = cursor.lastrowid, filename = filename)
        #print(msg)
    else:
        print(filename + 'not inserted')

    cnx.commit()

Conversion of pdf to text

In [5]:
def convert(infile, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    converter.close()
    text = output.getvalue()
    output.close
    return text

In [6]:
def get_record(filename):
    cmd = "SELECT * FROM pdf WHERE filename=%s"
    file = (filename,)
    cursor.execute(cmd, file)
    data = cursor.fetchall()
    if len(data) == 0:
        return False
    else:
        return True

**Extraction of PDF and storing in database**

In [6]:
def extractText(path):
    total = 0
    for filename in os.listdir(path):
        with open(os.path.join(path, filename), 'rb') as file:
            try:
                isPresent = False#get_record(filename)
                
                if isPresent==False:
                    extracted_text = convert(file)
                    insert_record(filename, extracted_text)
                    total = total+1
                else:
                    print(filename + " already inserted")
            except Error:
                print("error")
            
    print(total)

In [8]:
if __name__ == '__main__':
    make_connection()
    extractText("E:\save\malay\KudosWare\Data\Data")
    break_connection()
    localtime = time.localtime(time.time())
    print ("Local current time :", localtime)
    #os.system("shutdown /s /t 1")

72
Local current time : time.struct_time(tm_year=2020, tm_mon=11, tm_mday=18, tm_hour=17, tm_min=26, tm_sec=46, tm_wday=2, tm_yday=323, tm_isdst=0)
